# Обучение и тестирование TF-IDF

Импорт и загрузка библиотек

In [4]:
# %pip install -r requirements.txt

In [5]:
# %pip install gensim

In [6]:
import pandas as pd
import numpy as np
import string
import time
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

Загрузка датасета

In [7]:
df = pd.read_csv('cleared_dataset.csv')

Разделение данных

In [8]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

Инициализация TF-IDF из sklearn (стандартный токенизатор)

In [9]:
# Обучение будет проводиться по всем контекстам
tfidf_vectorizer_no_param_all = TfidfVectorizer()
tfidf_vectorizer_all = TfidfVectorizer(stop_words='english', ngram_range=(1, 2), max_df=0.8, sublinear_tf=True)

# Обучение будет проводиться по обучающим контекстам
tfidf_vectorizer_no_param_train = TfidfVectorizer()
tfidf_vectorizer_train = TfidfVectorizer(stop_words='english', ngram_range=(1, 2), max_df=0.85, sublinear_tf=True)

Инициализация TF-IDF из sklearn (кастомный токенизатор)

In [10]:
# Токенизатор с лемматизацией и удалением стоп-слов
def custom_tokenizer(text):
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english') + ['-', '-', '–','&'])
    tokens = word_tokenize(text.lower())
    tokens = [token for token in tokens if token not in string.punctuation and token not in stop_words]
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return lemmatized_tokens


tfidf_vectorizer_custom_all = TfidfVectorizer(tokenizer=custom_tokenizer, token_pattern=None, ngram_range=(1, 2), max_df=0.85, sublinear_tf=True)
tfidf_vectorizer_custom_train = TfidfVectorizer(tokenizer=custom_tokenizer, token_pattern=None, ngram_range=(1, 2), max_df=0.85, sublinear_tf=True)

Обучение TF-IDF со стандартным токенизатором

In [11]:
from scipy.sparse._matrix import spmatrix

# Обучение по всем контекстам
tfidf_vectorizer_no_param_all.fit(df["context"])

start_time = time.time()
tfidf_vectorizer_all.fit(df["context"])
end_time = time.time()
res_time = (end_time - start_time) * 1000
print(f'Время обучения tf-idf на {len(df)} записях: {res_time:.2f} ms')

# Обучение по обучающей выборке
tfidf_vectorizer_no_param_train.fit(train_df["context"])
tfidf_vectorizer_train.fit(train_df["context"])

Время обучения tf-idf на 11975 записях: 7370.70 ms


TfidfVectorizer(max_df=0.85, ngram_range=(1, 2), stop_words='english',
                sublinear_tf=True)

Обучение TF-IDF с кастомным токенизатором

In [12]:
# Обучение по всем контекстам
tfidf_vectorizer_custom_all.fit(df["context"])

# Обучение по обучающей выборке
tfidf_vectorizer_custom_train.fit(train_df["context"])

TfidfVectorizer(max_df=0.85, ngram_range=(1, 2), sublinear_tf=True,
                token_pattern=None,
                tokenizer=<function custom_tokenizer at 0x32a7d1f30>)

Векторизаця контекстов

In [13]:
# Все контексты
context_no_param_all = tfidf_vectorizer_no_param_all.transform(df['context'])
start_time = time.time()
context_all = tfidf_vectorizer_all.transform(df['context'])
end_time = time.time()
res_time = (end_time - start_time) * 1000 / len(df)
avg_tokens = context_all.getnnz(axis=1).mean()

print(f'Среднее время векторизации 1 контекста состояещего из {avg_tokens:.0f} токенов:', f'{res_time:.2f} ms')
context_custom_all = tfidf_vectorizer_custom_all.transform(df['context'])

# Тестовая выборка
context_no_param_test = tfidf_vectorizer_no_param_train.transform(test_df['context'])
context_test = tfidf_vectorizer_train.transform(test_df['context'])
context_custom_test = tfidf_vectorizer_custom_train.transform(test_df['context'])

Среднее время векторизации 1 контекста состояещего из 485 токенов: 0.29 ms


Анализ TF-IDF

Функция для тестирования и расчета точности

In [14]:
import numpy as np

def test_tf_idf_optimized(df, tfidf_vectorizer, tfidf_context, top_n=1):
    start_time = time.time()
    questions = df["question"].values
    true_context_indices = np.arange(len(df))
    question_vecs = tfidf_vectorizer.transform(questions)
    similarities_matrix = cosine_similarity(question_vecs, tfidf_context)
    top_indices_matrix = np.argsort(similarities_matrix, axis=1)[:, ::-1][:, :top_n]
    correct = sum(true_context_idx in top_indices
                  for true_context_idx, top_indices in zip(true_context_indices, top_indices_matrix))


    end_time = time.time()
    accuracy = correct / len(df)
    search_time = round((end_time - start_time) * 1000) / len(df)
    return accuracy, search_time

Построчное тестирование

In [15]:
def test_tf_idf(df, tfidf_vectorizer, tfidf_context, top_n=1) -> float:
    correct = 0
    total = len(df)
    log = []
    start_time = time.time()
    for i, row in df.iterrows():
        question = row["question"]
        true_context_idx = row.name
        true_context = row["context"]
        question_vec = tfidf_vectorizer.transform([question])
        similarities = cosine_similarity(question_vec, tfidf_context).flatten()
        top_indices = np.argsort(similarities)[::-1][:top_n]
        retrieved_contexts = df.iloc[top_indices]["context"].values
        temp_corr = 0
        if true_context_idx in top_indices:
            correct += 1
            temp_corr = 1
        max_similarity = max(similarities)
        log.append(
            {
                'num': i,
                'row_idx': true_context_idx,
                'correct': temp_corr,
                'max_similarity': max_similarity,
                'question': question,
                'true_context': true_context,
                'retrieved_contexts': retrieved_contexts
            }
        )
    end_time = time.time()
    accuracy = correct / total
    search_time = round((end_time - start_time) * 1000) / total
    return accuracy, log, search_time

Тестирование для выборки, обученной по всем контекстам

In [16]:
accuracy_no_param_all, search_time_1 = test_tf_idf_optimized(df, tfidf_vectorizer_no_param_all, context_no_param_all)
accuracy_all, search_time_2 = test_tf_idf_optimized(df, tfidf_vectorizer_all, context_all)
accuracy_custom_all, search_time_3 = test_tf_idf_optimized(df, tfidf_vectorizer_custom_all, context_custom_all)

print('Точность без параметров:', f'{(accuracy_no_param_all*100):.2f}%','Время на 1 вопрос:', f'{search_time_1:.2f} ms')
print('Точность с настройкой параметров:', f'{(accuracy_all*100):.2f}%','Время на 1 вопрос:', f'{search_time_2:.2f} ms')
print('Точность с кастомным токенизатором', f'{(accuracy_custom_all*100):.2f}%','Время на 1 вопрос:', f'{search_time_3:.2f} ms')

Точность без параметров: 69.27% Время на 1 вопрос: 0.86 ms
Точность с настройкой параметров: 88.34% Время на 1 вопрос: 0.36 ms
Точность с кастомным токенизатором 87.97% Время на 1 вопрос: 0.49 ms


Тестирование тестовой выборки, обученной на обучающей выборке

In [17]:
accuracy_no_param_test, search_time_4 = test_tf_idf_optimized(test_df, tfidf_vectorizer_no_param_train, context_no_param_test)
accuracy_test, search_time_5 = test_tf_idf_optimized(test_df, tfidf_vectorizer_train, context_test)
accuracy_custom_test, search_time_6 = test_tf_idf_optimized(test_df, tfidf_vectorizer_custom_train, context_custom_test)

print('Точность без параметров:', f'{(accuracy_no_param_test*100):.2f}%','Время на 1 вопрос:', f'{search_time_4:.2f} ms')
print('Точность с настройкой параметров:', f'{(accuracy_test*100):.2f}%','Время на 1 вопрос:', f'{search_time_5:.2f} ms')
print('Точность с кастомным токенизатором', f'{(accuracy_custom_test*100):.2f}%','Время на 1 вопрос:', f'{search_time_6:.2f} ms')

Точность без параметров: 73.74% Время на 1 вопрос: 0.13 ms
Точность с настройкой параметров: 85.51% Время на 1 вопрос: 0.08 ms
Точность с кастомным токенизатором 84.34% Время на 1 вопрос: 0.18 ms


### Итоги:
- Лучшая настройка tf-idf: `TfidfVectorizer(stop_words='english', ngram_range=(1, 2), max_df=0.85, sublinear_tf=True)` – Максимальная точность и скорость.
При данной настройке исключаются стоп слова, для векторизации используются 1-граммы и биграммы (тестировал с триграммами, результат ухудшается). Исключаются слова, встречающиеся в более чем 85% документов. Вместо прямого подсчета частоты термина используется формула 1 + log(tf), что помогает уменьшить влияние часто встречающихся слов.

- Лучшая точность для датаеста из 11975 записей при обучении на всех контекстах: 88.34%
- Лучшая точность для датаеста из 11975 записей при обучении на одних контекстах и тестирование на других: 85.51%

- Время обучения tf-idf на 11975 записях около 8 секунд
- Среднее время векторизации 1 контекста состояещего из 485 токенов: 0.3 ms
- Среднее время поиска 1 вопроса в 11975 контекстах: 0.3 ms

## sentence-transformers

In [ ]:
%pip install sentence-transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 KB 1.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 17.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.8/447.8 KB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 53.7 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 KB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.1/23.1 MB 41.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 39.4 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 MB 39.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.8/171.8 KB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.6/179.6 KB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 KB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB

In [19]:
from sentence_transformers import SentenceTransformer, util

# model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2', device='mps')
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2', device='mps')

In [20]:
embeded = model.encode(df['context'].tolist(), convert_to_tensor=True)

In [21]:
def test_sentence_transformer(df, embeded, model, top_n=3):
    correct = 0
    total = len(df)
    
    for i, row in df.iterrows():
        question = row['question']
        true_context = row['context']
        
        # Генерация эмбеддинга для вопроса
        question_embedding = model.encode(question, convert_to_tensor=True)
        
        # Вычисление косинусной схожести
        similarities = util.cos_sim(question_embedding, embeded)[0]
        
        # Индексы топ-N результатов
        top_indices = torch.topk(similarities, k=top_n).indices
        
        # Извлечение соответствующих контекстов
        retrieved_contexts = df.iloc[top_indices.cpu().numpy()]['context'].values
        
        # Проверка правильного ответа
        if true_context in retrieved_contexts:
            correct += 1
    
    # Расчет точности
    accuracy = correct / total
    return accuracy


accuracy = test_sentence_transformer(df, embeded, model, top_n=3)
print(f"Accuracy: {accuracy:.2f}")

NameError: name 'torch' is not defined

In [ ]:
top_k = 1
correct = 0
for i, row in df.iterrows():
    query = row['question']
    true_context_idx = row.name
    query_embedding = model.encode(query, convert_to_tensor=True)

    similarity_scores = model.similarity(query_embedding, embeded)[0]
    scores, indices = torch.topk(similarity_scores, k=top_k)


    if true_context_idx in indices:
            correct += 1
    
print(correct)

In [ ]:
print(correct/len(df))